# Pohlig-Hellman algorithm

We will perform the Pohlig-Hellman algorithm to compute $\log_{a} b$ in $\mathbf{Z}_{p}^*$, where $a = 11$, $b = 2018$, and $p = 15121$ is a prime number.

In [ ]:
from algorithms.factorization import totalFactorization
from algorithms.discreteLogarithm import babyStepGiantStep
from algorithms.modular import crt

a = 11
b = 2018
p = 15121

We notice that $p-1$ can be efficiently factorized. This allows the Pohlig-Hellman algorithm to break the problem into multiple smaller problems which can be solved, say, using the giant-step/baby-step algorithm.

In [ ]:
n = p - 1
f = totalFactorization(n)
f

We now perform the main loop of the algorithm. The idea is to compute discrete logarithms with base $(p-1)/q$ for each prime power factor $q^e$ of $p-1$ which give a $q$-ary representation of the sought result modulo $q^e$.

In [ ]:
ai = a^-1 % p
v = {}
for q, e in f.items():
    print("prime power factor: %d^%d" % (q, e))
    nq = n // q
    ap = pow(a, nq, p)
    print("a^(%d/%d) = %d" % (n, q, ap))
    bp, qp = b, 1
    c, k, x = 1, 0, 0
    for i in range(e):
        c = c * pow(ai, k, p) % p
        bp = pow(b*c, nq, p)
        print("c_%d = c_%d * a^-k_%d = %d" % (i, i-1, i-1, c))
        print("b_%d = (b_%d*c_%d)^(%d/%d^%d) = %d" %
                                            (i, i-1, i, n, q, i+1, bp))
        print("z = log_%d(%d) mod %d, order = %d" % (ap, bp, p, q))
        k = (babyStepGiantStep(ap, bp, p, order = q) % q) * qp
        x += k
        print("k_%d = z*q^%d = %d" % (i, i, k))
        qp *= q
        nq //= q
    v[qp] = x
v

This gives a system of congruences with pairwise coprime moduli, which can then be solved using the Chinese remainder theorem.

In [ ]:
x = crt(v)
x

Let us check that we get the correct answer.

In [ ]:
pow(a, x, p)